In [1]:
import cv2
import numpy as np
import os

#### Load Images

This function load_images_from_folder takes a folder path as input and iterates through each file in the folder. It uses cv2.imread to read each image file. If the image is successfully read (i.e., img is not None), it is appended to the images list. Finally, it returns the list of images.

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

#### Preprocessing an Image

This function converts an image to grayscale, applies Gaussian blur to reduce noise, and then equalizes the histogram of the image. These steps are typical preprocessing tasks to make feature extraction more effective.

In [3]:
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    equalized = cv2.equalizeHist(blurred)
    return equalized

#### Extracting Features from an Image

This function initializes a SIFT (Scale-Invariant Feature Transform) descriptor, then detects keypoints and computes descriptors for the input image. The keypoints are locations in the image that are considered to have unique features, and descriptors are the features at those keypoints.

In [4]:
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

#### Matching Features Between Two Images

This function uses the FLANN (Fast Library for Approximate Nearest Neighbors) algorithm to match descriptors between two images. knnMatch finds the two nearest neighbors for each descriptor. The function then filters these to retain only good matches based on the distance between the matched descriptors.

In [5]:
def feature_matching(descriptors1, descriptors2):
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = [m for m, n in matches if m.distance < 0.7*n.distance]
    return good_matches

#### Classifying a Tumor in an Image

This function processes the test image, then iterates through each folder of training images, corresponding to different tumor types. It processes and extracts features from each training image, matches these features with those of the test image, and calculates the average number of matches per folder. The folder (tumor type) with the highest average match count is considered the classification result.

In [6]:
def classify_tumor(test_image, train_folders):
    preprocessed_test_image = preprocess_image(test_image)
    print(train_folders)
    _, test_descriptors = extract_features(preprocessed_test_image)
    max_matches = 0
    tumor_type = None

    for folder in train_folders:
        total_matches = 0
        train_images = load_images_from_folder(folder)
        for train_image in train_images:
            preprocessed_train_image = preprocess_image(train_image)
            _, train_descriptors = extract_features(preprocessed_train_image)
            matches = feature_matching(test_descriptors, train_descriptors)
            total_matches += len(matches)

        avg_matches = total_matches / len(train_images)
        print(f"Average matches for {os.path.basename(folder)}: {avg_matches}")

        if avg_matches > max_matches:
            max_matches = avg_matches
            tumor_type = os.path.basename(folder)

    return tumor_type

#### Setting Up Paths and Classifying a Test Image

This part sets up the file paths for the training folders and the test image, reads the test image using OpenCV, and calls classify_tumor with the test image and the training folders. It then prints out the classification result, indicating the type of tumor identified in the test image.

Since it takes too long to test all the test data, it was tested for only 1 image so that you can test it easily.

In [7]:
train_base_folder = r'C:\Users\bltkb\Desktop\spring 24\Bil 468 Proje\Method1\dataset1\Training'
train_folders = [os.path.join(train_base_folder, f) for f in ['meningioma_tumor','glioma_tumor', 'no_tumor', 'pituitary_tumor']]

test_image_path = r'C:\Users\bltkb\Desktop\spring 24\Bil 468 Proje\Method1\dataset1\Testing\meningioma_tumor\image.jpg'

test_image = cv2.imread(test_image_path)
tumor_type = classify_tumor(test_image, train_folders)
print(f"The test image is classified as: {tumor_type}")

['C:\\Users\\bltkb\\Desktop\\spring 24\\Bil 468 Proje\\Method1\\Training\\meningioma_tumor', 'C:\\Users\\bltkb\\Desktop\\spring 24\\Bil 468 Proje\\Method1\\Training\\glioma_tumor', 'C:\\Users\\bltkb\\Desktop\\spring 24\\Bil 468 Proje\\Method1\\Training\\no_tumor', 'C:\\Users\\bltkb\\Desktop\\spring 24\\Bil 468 Proje\\Method1\\Training\\pituitary_tumor']


Average matches for meningioma_tumor: 4.54683698296837
Average matches for glioma_tumor: 4.953389830508475
